<a href="https://colab.research.google.com/github/superproger228/coursework/blob/main/PyTorch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import pandas as pd 
import torch.nn as nn 
from torch.utils.data import random_split, DataLoader, TensorDataset 
import torch.nn.functional as F 
import numpy as np 
import torch.optim as optim 
from torch.optim import Adam
import seaborn as sns

In [2]:
df = sns.load_dataset("iris")
df.rename(columns = {'species':'Iris_Type'}, inplace = True )
df.head() 
#df['Iris_Type'].value_counts()

,sepal_length,sepal_width,petal_length,petal_width,Iris_Type
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
labels = {'setosa':0, 'versicolor':1, 'virginica':2} 
df['IrisType_num'] = df['Iris_Type']  
df.IrisType_num = [labels[item] for item in df.IrisType_num]  
 
input = df.iloc[:,:-2]           
print(input, '\n')   
output = df.loc[:, 'IrisType_num']   
print(output)

     sepal_length  sepal_width  petal_length  petal_width
0             5.1          3.5           1.4          0.2
1             4.9          3.0           1.4          0.2
2             4.7          3.2           1.3          0.2
3             4.6          3.1           1.5          0.2
4             5.0          3.6           1.4          0.2
..            ...          ...           ...          ...
145           6.7          3.0           5.2          2.3
146           6.3          2.5           5.0          1.9
147           6.5          3.0           5.2          2.0
148           6.2          3.4           5.4          2.3
149           5.9          3.0           5.1          1.8

[150 rows x 4 columns] 

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: IrisType_num, Length: 150, dtype: int64


In [4]:
input = torch.Tensor(input.to_numpy())     
output = torch.tensor(output.to_numpy())        
data = TensorDataset(input, output)    

In [5]:
train_batch_size = 10        
number_rows = len(input) 
test_split = int(number_rows*0.2)  
validate_split = int(number_rows*0.1) 
train_split = number_rows - test_split - validate_split   #70% - train
train_set, validate_set, test_set = random_split( 
    data, [train_split, validate_split, test_split])    
 
train_loader = DataLoader(train_set, batch_size = train_batch_size, shuffle = True) 
validate_loader = DataLoader(validate_set, batch_size = 1) 
test_loader = DataLoader(test_set, batch_size = 1)

In [6]:
input_size = list(input.shape)[1]   # = 4. 
learning_rate = 0.01 
output_size = len(labels)           
N = 24

class Network(nn.Module): 
   def __init__(self, input_size, output_size): 
       super(Network, self).__init__() 
        
       self.layer1 = nn.Linear(input_size, N) 
       self.layer2 = nn.Linear(N, N) 
       self.layer3 = nn.Linear(N, output_size) 

   def forward(self, x): 
       x1 = F.relu(self.layer1(x)) 
       x2 = F.relu(self.layer2(x1)) 
       x3 = self.layer3(x2) 
       return x3 
 
model = Network(input_size, output_size)

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

def saveModel(): 
    torch.save(model.state_dict(), " ") 

In [18]:
def train(num_epochs): 
    best_accuracy = 0.0 
     
    for epoch in range(1, num_epochs+1): 
        running_train_loss = 0.0 
        running_accuracy = 0.0 
        running_vall_loss = 0.0 
        total = 0 
 
        for data in train_loader: 
            inputs, outputs = data   
            optimizer.zero_grad()   
            predicted_outputs = model(inputs)   
            train_loss = loss_fn(predicted_outputs, outputs)   
            train_loss.backward()   
            optimizer.step()        
            running_train_loss +=train_loss.item()  
        
        train_loss_value = running_train_loss/len(train_loader) 
      
        with torch.no_grad(): 
            model.eval() 
            for data in validate_loader: 
               inputs, outputs = data 
               predicted_outputs = model(inputs) 
               val_loss = loss_fn(predicted_outputs, outputs) 
                       
               _, predicted = torch.max(predicted_outputs, 1) 
               running_vall_loss += val_loss.item()  
               total += outputs.size(0) 
               running_accuracy += (predicted == outputs).sum().item() 
 
        val_loss_value = running_vall_loss/len(validate_loader) 
                
        accuracy = (100 * running_accuracy / total)     
 
        if accuracy > best_accuracy: 
            saveModel()
            best_accuracy = accuracy 
         
        print('batch', epoch, 'Training Loss: %.2f' %train_loss_value, 'Validation Loss: %.2f' %val_loss_value, 'Accuracy %d %%' % (accuracy))

In [19]:
num_epochs = 10
train(num_epochs) 

batch 1 Training Loss: 0.07 Validation Loss: 0.03 Accuracy 100 %
batch 2 Training Loss: 0.12 Validation Loss: 0.00 Accuracy 100 %
batch 3 Training Loss: 0.11 Validation Loss: 0.01 Accuracy 100 %
batch 4 Training Loss: 0.08 Validation Loss: 0.01 Accuracy 100 %
batch 5 Training Loss: 0.09 Validation Loss: 0.07 Accuracy 93 %
batch 6 Training Loss: 0.07 Validation Loss: 0.05 Accuracy 100 %
batch 7 Training Loss: 0.17 Validation Loss: 0.01 Accuracy 100 %
batch 8 Training Loss: 0.18 Validation Loss: 0.07 Accuracy 93 %
batch 9 Training Loss: 0.15 Validation Loss: 0.04 Accuracy 100 %
batch 10 Training Loss: 0.17 Validation Loss: 0.05 Accuracy 100 %


In [20]:
def test(): 
     
    running_accuracy = 0 
    total = 0 
 
    with torch.no_grad(): 
        for data in test_loader: 
            inputs, outputs = data 
            outputs = outputs.to(torch.float32) 
            predicted_outputs = model(inputs) 
            _, predicted = torch.max(predicted_outputs, 1) 
            total += outputs.size(0) 
            running_accuracy += (predicted == outputs).sum().item() 
 
        print('Accuracy: %d %%' % (100 * running_accuracy / total))    
 

In [21]:
test() 

Accuracy: 93 %
